# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
import requests
url = 'http://api.citybik.es/v2/networks/bixi-montreal'
#From the city bike website, we are shown that the API endpoint 

response = requests.get(url)

if response.status_code == 200:
    print("Success! URL is working.")
elif response.status_code == 404:
    print("Not Found.")
# parse the JSON content. The requests library provides a built-in .json() method that parses the JSON content of a response 
# into a Python dictionary or list, making it easy to work with.
response_data = response.json()


Success! URL is working.


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [3]:
response_data.keys() #Showed me that network is the main key
response_data['network'].keys() #Showed me the keys within the network key
response_data['network']['stations'] #Showed me the first station in the list of stations
response_data['network']['stations'][0]['empty_slots'] #Showed me the number of empty slots at the first station
response_data['network']['stations'][0]['free_bikes'] #Showed me the number of free bikes at the first station
response_data['network']['stations'][0]['latitude'] #Showed me the latitude of the first station
response_data['network']['stations'][0]['longitude'] #Showed me the longitude of the first station
response_data['network']['stations'][0]['name'] #Showed me the name of the first station
response_data['network']['stations'][0]['id'] #Showed me the id of the first station

stations_subset_response_data = response_data['network']['stations'] #Created a variable for the list of stations
stations_subset_response_data




[{'empty_slots': 12,
  'extra': {'ebikes': 0,
   'has_ebikes': True,
   'last_updated': 1701706499,
   'payment': ['key', 'creditcard'],
   'payment-terminal': True,
   'renting': 1,
   'returning': 1,
   'slots': 15,
   'uid': '740'},
  'free_bikes': 3,
  'id': '36c6491aa1b52e5ef7005f984738de27',
  'latitude': 45.516926210319546,
  'longitude': -73.56425732374191,
  'name': "Gare d'autocars de Montréal (Berri / Ontario)",
  'timestamp': '2023-12-04T16:17:16.707000Z'},
 {'empty_slots': 13,
  'extra': {'ebikes': 0,
   'has_ebikes': True,
   'last_updated': 1701705554,
   'payment': ['key', 'creditcard'],
   'payment-terminal': True,
   'renting': 1,
   'returning': 1,
   'slots': 22,
   'uid': '330'},
  'free_bikes': 9,
  'id': '65f5707d61306518123222b4680bc282',
  'latitude': 45.55321884238814,
  'longitude': -73.53978216648102,
  'name': 'Marché Maisonneuve',
  'timestamp': '2023-12-04T16:17:16.642000Z'},
 {'empty_slots': 12,
  'extra': {'ebikes': 0,
   'has_ebikes': True,
   'last_up

Put your parsed results into a DataFrame.

In [47]:
import pandas as pd
bixi_stations_df = pd.json_normalize(stations_subset_response_data) #Created a dataframe from the list of stations
# .json_normalize() method to flatten the nested JSON data into a table !!!
bixi_stations_df.head() 




,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid
0,12,3,36c6491aa1b52e5ef7005f984738de27,45.516926,-73.564257,Gare d'autocars de Montréal (Berri / Ontario),2023-12-04T16:17:16.707000Z,0,True,1701706499,"[key, creditcard]",True,1,1,15,740
1,13,9,65f5707d61306518123222b4680bc282,45.553219,-73.539782,Marché Maisonneuve,2023-12-04T16:17:16.642000Z,0,True,1701705554,"[key, creditcard]",True,1,1,22,330
2,12,17,0e58810dea9e4869279089cb8576642f,45.526890,-73.572640,Rachel / de Brébeuf,2023-12-04T16:17:16.738000Z,0,True,1701704549,"[key, creditcard]",True,1,1,29,144
3,9,6,eeda0f4010f34ca2199f3585a76fef66,45.553400,-73.662255,Bibliothèque d'Ahuntsic (Lajeunesse / Fleury),2023-12-04T16:17:16.528000Z,0,True,1701706510,"[key, creditcard]",True,1,1,15,797
4,15,19,8140fc3c3486abf913b0bd9211926f2b,45.559842,-73.615447,Cité des Arts du Cirque (Paul Boutet / des Reg...,2023-12-04T16:17:16.641000Z,0,True,1701697364,"[key, creditcard]",True,1,1,34,453


In [48]:
columns_to_drop = list(bixi_stations_df.columns[6:]) #the columns I wanted to drop

#Dropped the columns not needed for this analysis
bixi_stations_df.drop(columns_to_drop, axis=1, inplace=True)
bixi_stations_df.head()

,empty_slots,free_bikes,id,latitude,longitude,name
0,12,3,36c6491aa1b52e5ef7005f984738de27,45.516926,-73.564257,Gare d'autocars de Montréal (Berri / Ontario)
1,13,9,65f5707d61306518123222b4680bc282,45.553219,-73.539782,Marché Maisonneuve
2,12,17,0e58810dea9e4869279089cb8576642f,45.526890,-73.572640,Rachel / de Brébeuf
3,9,6,eeda0f4010f34ca2199f3585a76fef66,45.553400,-73.662255,Bibliothèque d'Ahuntsic (Lajeunesse / Fleury)
4,15,19,8140fc3c3486abf913b0bd9211926f2b,45.559842,-73.615447,Cité des Arts du Cirque (Paul Boutet / des Reg...


In [49]:
bixi_stations_df.shape
bixi_stations_df.info()
# Maybe convert latitude and longitude to strings for API call 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  160 non-null    int64  
 1   free_bikes   160 non-null    int64  
 2   id           160 non-null    object 
 3   latitude     160 non-null    float64
 4   longitude    160 non-null    float64
 5   name         160 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 7.6+ KB


In [50]:
bixi_stations_df.isnull().sum() #No null values

empty_slots    0
free_bikes     0
id             0
latitude       0
longitude      0
name           0
dtype: int64

In [52]:
bixi_stations_df.duplicated().sum() #No duplicates


0

In [53]:
bixi_stations_df.to_csv('bixi_stations_df.csv')